In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import Stock_Func
from sklearn import linear_model, metrics, model_selection
import random

In [2]:
train_data = pd.read_csv('train.csv', sep=',', index_col=0)
test_data = pd.read_csv('test.csv', sep=',', index_col=0)

## Add "RSV" to train_data & test_data
RSV = １００＊（今日收盤價－最近９天的最低價）／（最近９天的最高價－最近９天的最低價）
## Add "Kvalue" to train_data & test_data
Kvalue = 前一天的Kvalue＊２／３　＋　當日RSV＊１／３
## Add "Dvalue" to train_data & test_data
Dvalue = 前一天的Dvalue＊２／３　＋　當日Kvalue＊１／３
## Add "RisePredict" to train_data & test_data
RisePredict：明天是否會漲（漲：１　跌：０）

In [3]:
test_data = Stock_Func.NewFeature(test_data)
train_data = Stock_Func.NewFeature(train_data)

## Add up&down to  train_data & test_data
up&down：今日漲跌幅（今日收盤價－昨日收盤價）

In [4]:
test_data = Stock_Func.Up_Down(test_data)
train_data = Stock_Func.Up_Down(train_data)

## Add RSI6 to train_data & test_data
RSI6 = １００＊６日內的漲幅／（６日內的漲幅＋６日內的跌幅）
## Add RSI12 to train_data & test_data
RSI12 = １００＊１２日內的漲幅／（１２日內的漲幅＋１２日內的跌幅）


In [5]:
test_data = Stock_Func.AddRSI6(test_data)
train_data = Stock_Func.AddRSI6(train_data)
test_data = Stock_Func.AddRSI12(test_data)
train_data = Stock_Func.AddRSI12(train_data)

## Drop the row 0-11 and the last row
因為前12列缺少RSI12；最後一列缺少RisePredict

In [6]:
train_data = train_data.drop(train_data.index[[0,1,2,3,4,5,6,7,
                                               8,9,10,11,
                                               train_data.shape[0]-1]])
test_data = test_data.drop(test_data.index[[0,1,2,3,4,5,6,7,
                                            8,9,10,11,
                                            test_data.shape[0]-1]])

## Add "KD_variation" column to train_data & test_data
#### KD_variation：
###### 黃金交叉：若Kvalue由下往上突破Dvalue，表示該股上漲機率高
###### 死亡交叉：若Kvalue由上往下跌破Dvalue，表示該股下跌機率高
###### 黃金交叉（２）,死亡交叉（－２）,Kvalue和Dvalue皆向上（１）,Kvalue和Dvalue皆向下（－１）,Kvalue和Dvalue連續兩天相等（０）
## Add "KD_strength" column to train_data & test_data
#### KD_strength：
###### Kvalue和Dvalue皆大於８０：該股處於強勢期，上漲機率高
###### Kvalue和Dvalue皆小於２０：該股處於弱勢期，下跌機率高
###### Kvalue和Dvalue皆大於８０（１）,Kvalue和Dvalue皆小於２０（－１）,其他狀況（０）

In [7]:
train_data = Stock_Func.KD_variation(train_data)
test_data = Stock_Func.KD_variation(test_data)
train_data = Stock_Func.KD_strength(train_data)
test_data = Stock_Func.KD_strength(test_data)

## Add "RSI_variation" column to train_data & test_data
#### RSI_variation：
###### 黃金交叉：若RSI6由下往上突破 RSI12，表示該股上漲機率高
###### 死亡交叉：若RSI6由上往下跌破 RSI12，表示該股下跌機率高
###### 黃金交叉（２）,死亡交叉（－２）,RSI6和RSI12皆向上（１）,RSI6和RSI12皆向下（－１）,RSI6和RSI12連續兩天相等（０）

In [8]:
train_data = Stock_Func.RSI_variation(train_data)
test_data = Stock_Func.RSI_variation(test_data)

In [9]:
train_data

,Open Price,Close Price,High Price,Low Price,Volume,RSV,Kvalue,Dvalue,RisePredict,Up_Down,RSI6,RSI12,KD_variation,KD_strength,RSI_variation
Date,,,,,,,,,,,,,,,
21-Jan-2009,806.77,840.24,841.72,804.30,6467829760,33.508755,22.372940,28.135318,0,35.02,37.292584,27.146565,0,0,0
22-Jan-2009,839.74,827.50,839.74,811.29,5843829760,26.171737,23.639206,26.636614,1,-12.74,32.877136,26.055292,-1,0,1
23-Jan-2009,822.16,831.95,838.61,806.07,5832160256,40.153222,29.143878,27.472369,1,4.45,44.900105,25.049779,2,0,1
26-Jan-2009,832.50,836.57,852.53,827.69,6039940096,47.093285,35.127014,30.023917,1,4.62,46.683933,30.797896,1,0,1
27-Jan-2009,837.30,845.71,850.45,835.40,5353260032,90.178174,53.477400,37.841745,1,9.14,48.011184,33.022170,1,0,1
28-Jan-2009,845.73,874.09,877.86,845.73,6199179776,100.000000,68.984934,48.222808,0,28.38,86.497085,45.884378,1,0,1
29-Jan-2009,868.89,845.14,868.89,844.15,5067060224,57.964281,65.311383,53.918999,0,-28.95,52.775261,43.914729,1,0,1
30-Jan-2009,845.69,825.88,851.66,821.67,5350580224,29.998548,53.540438,53.792812,0,-19.26,49.145570,39.758176,-2,0,1
02-Feb-2009,823.09,825.44,830.78,812.87,5673269760,0.000000,35.693625,47.759750,1,-0.44,46.414803,45.603889,-1,0,1


# Train the model
## Use the original data to predict the up and down.
#### Attributes:	"Open Price"	"Close Price"	"Low Price"	  "High Price"

In [10]:
drop_list = ['Volume','RSV','Up_Down','RSI6','RSI12','RSI_variation',
            'Kvalue','Dvalue','KD_variation','KD_strength']
_train_data = train_data
_test_data = test_data
_train_data = train_data.drop(drop_list, axis = 1)
_test_data = test_data.drop(drop_list, axis = 1) 

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x_train = _train_data.loc[:, _train_data.columns != 'RisePredict']
y_train = _train_data['RisePredict']
x_test = _test_data.loc[:, _test_data.columns != 'RisePredict']
y_test = _test_data['RisePredict']
clf = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='multinomial').fit(x_train, y_train)

In [12]:
pre_result = clf.predict(x_test).tolist()
print(classification_report(y_test, pre_result))
print(clf.score(x_test, y_test))

              precision    recall  f1-score   support

           0       0.56      0.04      0.08       117
           1       0.51      0.97      0.67       122

   micro avg       0.51      0.51      0.51       239
   macro avg       0.53      0.50      0.37       239
weighted avg       0.53      0.51      0.38       239

0.5146443514644351


## Use the K/D value to predict the up and down.
#### Attributes:	"High Price"	"Kvalue"	"Dvalue"	"KD_variation"	"KD_strength"

In [13]:
drop_list = ['Open Price','Close Price','Low Price','Volume','RSV','Up_Down','RSI6','RSI12','RSI_variation']
_train_data = train_data
_test_data = test_data
_train_data = train_data.drop(drop_list, axis = 1)
_test_data = test_data.drop(drop_list, axis = 1) 

In [14]:
x_train = _train_data.loc[:, _train_data.columns != 'RisePredict']
y_train = _train_data['RisePredict']
x_test = _test_data.loc[:, _test_data.columns != 'RisePredict']
y_test = _test_data['RisePredict']
clf = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='multinomial').fit(x_train, y_train)

In [15]:
pre_result = clf.predict(x_test).tolist()
print(classification_report(y_test, pre_result))
print(clf.score(x_test, y_test))

              precision    recall  f1-score   support

           0       0.58      0.15      0.24       117
           1       0.52      0.89      0.66       122

   micro avg       0.53      0.53      0.53       239
   macro avg       0.55      0.52      0.45       239
weighted avg       0.55      0.53      0.46       239

0.5313807531380753


## Use the RSI6/RSI12 value to predict the up and down.
#### Attributes:	"High Price"	"RSI12"		"RSI_variation"

In [16]:
drop_list = ['Open Price','Close Price','Low Price','Volume','RSV',
             'RSI6','Up_Down','Kvalue','Dvalue','KD_variation','KD_strength']
_train_data = train_data
_test_data = test_data
_train_data = train_data.drop(drop_list, axis = 1)
_test_data = test_data.drop(drop_list, axis = 1) 

In [17]:
x_train = _train_data.loc[:, _train_data.columns != 'RisePredict']
y_train = _train_data['RisePredict']
x_test = _test_data.loc[:, _test_data.columns != 'RisePredict']
y_test = _test_data['RisePredict']
clf = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='multinomial').fit(x_train, y_train)

In [18]:
pre_result = clf.predict(x_test).tolist()
print(classification_report(y_test, pre_result))
print(clf.score(x_test, y_test))

              precision    recall  f1-score   support

           0       0.42      0.04      0.08       117
           1       0.51      0.94      0.66       122

   micro avg       0.50      0.50      0.50       239
   macro avg       0.46      0.49      0.37       239
weighted avg       0.46      0.50      0.37       239

0.502092050209205
